# **COVID-19 Dashboard (Brasil)**

## 1\. Contexto

Segundo a Organização Mundial da Saúde (OMS), o Brasil é o país com o maior número de casos de COVID-19 na América do Sul. Esse problema é ainda mais grave entre os grupos menos favorecidos, devido às desigualdades sociais. A pandemia de COVID-19 representa uma crise global, destacando a importância da análise de dados para apoiar o planejamento e a tomada de decisões. Diante disso, este projeto busca analisar os casos e óbitos por COVID-19 no Brasil através de um dashboard interativo.

### 1.1\. TLDR

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/41f0e068-86f4-4a4b-8074-8c9d028beb50)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

### 3.1\. Extração dos casos de morte

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

Os dados estão em arquivos separados. Portanto, vamos iterar dentro de um for com períodos definidos e extraí-los.

In [3]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [4]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

Dentro de um for, vamos selecionar as colunas de interesse, e extraír apenas os dados para o Brasil.

In [5]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], axis= 0, ignore_index=True)

### 3.2\. Extração dos dados de vacinação

Vamos processar os dados de **vacinação** da universidade de Oxford.

In [19]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

Selecionando colunas de interesse e dados do Brasil.

In [21]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

## 4\. Transformação

### 4.1\. Tranformação dos casos de morte

In [10]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [11]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Chaves temporais

In [12]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

População estimada do estado

In [14]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

 Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado

In [15]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:    # Se a divisão for menor que 0.75 (correção 0.85)
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state],axis=0, ignore_index=True)

cases = cases_
cases_ = None

type casting das colunas

In [16]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [17]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

### 4.2\. Transformação dos dados de vacina

Trataremos dados faltantes preenchendo os "buracos" com o valor anterior válido mais próximo.

In [ ]:
vaccines = vaccines.fillna(method='ffill')

Filtrando a coluna date para que ambas estejam na mesma forma.

In [22]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Alterando os nomes das colunas.

In [31]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

Chaves temporais

In [32]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

Dados relativos

In [33]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

type casting das colunas.

In [34]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [35]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

## 5\. Carregamento

### 5.1\. Salvando os dados de casos de covid

In [ ]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

### 5.1\. Salvando os dados de casos de vacina

In [ ]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

link: https://lookerstudio.google.com/reporting/41f0e068-86f4-4a4b-8074-8c9d028beb50

## 6\. Storytelling

# **COVID-19 DASHBOARD (BRAZIL)**

## Visão Geral
Este projeto utilizou dados compilados de casos da COVID-19 pelo Centro de Ciência de Sistemas e Engenharia da Universidade Americana John Hopkins ([link](https://www.jhu.edu)), e dados de vacinação contra a COVID-19 pelo Nosso Mundo em Dados (Our World in Data) de Oxford [link](https://www.ox.ac.uk). Os dados passaram pelo processo de extração e tranformação para que fossem visualizados em um dashboard criado no Google Data Studio.

---

## Objetivo
O principal objetivo deste projeto foi analisar o número de mortes e de pessoas vacinadas no ano de 2021, e isso inclui a primeira, segunda e terceira dose, além de médias móveis e tendências.

---

## Metodologia

1. **Coleta de Dados:**
   - Dados de Casos da COVID-19 - Centro de Ciência de Sistemas e Engenharia da Universidade Americana John Hopkins ([link](https://www.jhu.edu)).
   - Dados de vacinação contra a COVID-19 - Oxford [link](https://www.ox.ac.uk).

   
2. **Pré-processamento:**
   - Organizei os dados tanto da base de dados de casos quanto de vacina de modo a deixar que ambas se complementem.
   
3. **Análise:**
   - A análise interativa dos dados foi feita através de um dashboard gerado no Google Data Studio.

---

## Conclusão

Os resultados mostram os aumentos de diminuições de casos de CODID-19 e de mortes pela mesma, respectivamente, assim como a média móvel e a tendência. 

